#### Ciencia de datos - Primavera 2024
##### Integrantes: Magdalena Cobb, Pedro García Vassallo, Marcos Olavarría
### Trabajo Práctico 4

# 1. LECTURA

In [1]:
#Paquetes y librerías a usar
# CHEQUEO DE CAMBIOS
#!pip install openpyxl
#!pip install statsmodels

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
#from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Lectura del documento
indi_2024 = pd.read_excel('C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/usu_individual_T124.xlsx') 
indi_2004 = pd.read_stata('C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/Individual_t104.dta')
hoga_2024 = pd.read_excel('C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/usu_hogar_T124.xlsx')
hoga_2004 = pd.read_stata('C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/Hogar_t104.dta')

# Maggie: C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/
# Peter: 'C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/
# Marck: C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/


# 2. Selección de GBA y merge

a. Eliminen todas las observaciones que no corresponden a los
aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos
Aires, y unan ambos trimestres en una sola base.

In [3]:
#Filtramos las bases para tener solo Buenos Aires
ind_2024 = indi_2024[(indi_2024['AGLOMERADO'] == 33) | (indi_2024['AGLOMERADO'] == 32)]
hog_2024 = hoga_2024[(hoga_2024['AGLOMERADO'] == 33) | (hoga_2024 ['AGLOMERADO'] == 32 )]

ind_2004 = indi_2004[(indi_2004['aglomerado'] == 'Ciudad de Buenos Aires') | (indi_2004 ['aglomerado'] == 'Partidos del GBA' )]
hog_2004 = hoga_2004[(hoga_2004['aglomerado'] == 'Ciudad de Buenos Aires') | (hoga_2004 ['aglomerado'] == 'Partidos del GBA' )]

ind_2004.columns = ind_2004.columns.str.upper()
hog_2004.columns = hog_2004.columns.str.upper()


In [4]:
#Eliminación de columnas redundantes (preguntas hechas solo para ocupados o desocupados)
redundantes_ind04 = [
    "CAT_INAC",
    "PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3",
    "PP11A", "PP11B_COD", "PP11B1", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA", "PP11C", "PP11C99", "PP11D_COD",
    "PP11G_ANO", "PP11G_MES", "PP11G_DIA", "PP11L", "PP11L1", "PP11M", "PP11N", "PP11O", "PP11P", "PP11Q", "PP11R", 
    "PP11S", "PP11T", "PP02C1", "PP02C2", "PP02C3", "PP02C4", "PP02C5", "PP02C6", "PP02C7", "PP02C8", "PP02E", 
    "PP02H", "PP02I", "CAT_OCUP", "PP03C", "PP03D", "PP3E_TOT", "PP3F_TOT", "PP03G", "PP03H", "PP03I", "PP03J", 
    "INTENSI", "PP04A", "PP04B_COD", "PP04B1", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", "PP04C", 
    "PP04C99", "PP04D_COD", "PP04G", "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP05C_1", "PP05C_2", "PP05C_3", 
    "PP05E", "PP05F", "PP05H", "PP06A", "PP06C", "PP06E", "PP06H", "PP07A", "PP07C", "PP07D", "PP07F1", "PP07F2", 
    "PP07F3", "PP07F4", "PP07F5", "PP07G1", "PP07G2", "PP07G3", "PP07G4", "PP07G_59", "PP07H", "PP07I", "PP07J", 
    "PP07K", "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR", "TOT_P12", "PJ1_1", 
    "PJ2_1", "IDIMPP"
]

redundantes_hog04 = [
    "II3",
    "II3_1",
    "II6",
    "II6_1"
]

redundantes_ind24 = [
    "PP02C1", "PP02C2", "PP02C3", "PP02C4", "PP02C5", "PP02C6", "PP02C7", "PP02C8",
    "PP02E", "PP02H", "PP02I", "PP10A", "PP10C", "PP10D", "PP10E", "PP11A",
    "PP11B_COD", "PP11B1", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA", "PP11C", 
    "PP11C99", "PP11D_COD", "PP11G_ANO", "PP11G_MES", "PP11G_DIA", "PP11L", "PP11L1",
    "PP11M", "PP11N", "PP11O", "PP11P", "PP11Q", "PP11R", "PP11S", "PP11T",
    "PP03C", "PP03D", "PP3E_TOT", "PP3F_TOT", "PP03G", "PP03H", "PP03I", "PP03J",
    "INTENSI", "PP04A", "PP04B_COD", "PP04B1", "PP04B2", "PP04B3_MES", "PP04B3_ANO",
    "PP04C", "PP04C99", "PP04D_COD", "PP04G", "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA",
    "PP05C_1", "PP05C_2", "PP05C_3", "PP05E", "PP05F", "PP05H", "PP06A", "PP06C",
    "PP06D", "PP06E", "PP06H", "PP07A", "PP07C", "PP07D", "PP07E", "PP07F1", "PP07F2",
    "PP07F3", "PP07F4", "PP07F5", "PP07G1", "PP07G2", "PP07G3", "PP07G4", "PP07G_59",
    "PP07H", "PP07I", "PP07J", "PP07K", "PP08D1", "PP08D4", "PP08F1", "PP08F2", 
    "PP08J1", "PP08J2", "PP08J3", "PP09A", "PP09A_ESP", "PP09B", "PP09C", "PP09C_ESP",
    "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR",
    "PONDIIO", "TOT_P12", "P47T", "DECINDR", "IDECINDR", "RDECINDR", "GDECINDR", 
    "PDECINDR", "ADECINDR", "PONDII" 
]

redundantes_hog24 = [
    "II3",
    "II3_1",
    "II6",
    "II6_1"
]



In [5]:
ind_2004 = ind_2004.drop(columns = redundantes_ind04)
ind_2024 = ind_2024.drop(columns = redundantes_ind24)
hog_2004 = hog_2004.drop(columns = redundantes_hog04)
hog_2024 = hog_2024.drop(columns = redundantes_hog24)

In [6]:
lista_con = ['ANO4', 'TRIMESTRE', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'ITF', 'DECIFR', 'IDECIFR', 
             'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR','PDECCFR', 'ADECCFR']

lista_con2 = ['ANO4', 'TRIMESTRE', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'ITF', 'DECIFR', 'IDECIFR', 'RDECIFR',
              'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'ADECCFR','PDECCFR' ,'PONDIH']

ind_2004 = ind_2004.drop (columns = (lista_con))
ind_2024 = ind_2024.drop(columns = (lista_con2))

In [7]:
combined_04 = pd.merge(ind_2004, hog_2004, on=['CODUSU', 'NRO_HOGAR'], how='outer')
combined_24 = pd.merge(ind_2024,hog_2024,on=["CODUSU","NRO_HOGAR"], how = 'outer')

In [ ]:
lista_columnas = combined_24.columns.tolist()
lista_columnas

In [ ]:
combined_24

In [10]:
combined_24 = combined_24.drop(columns = "PDECCFR")

In [ ]:
combined_24

# 2. Limpieza
Limpien la base de datos tomando criterios que hagan sentido. Explicar
cualquier decisión como el tratamiento de valores faltantes (missing
values), extremos (outliers), o variables categóricas. Justifique sus
decisiones.

In [73]:
# Se usa base de datos:
# 2024: 
# - combined_24

# 2004:
# - combined_04

##### Limpieza de datos

In [ ]:
# Limpieza de datos tomando criterios que hagan sentido
combined_24
print(combined_24.describe())

In [40]:
# Verificar si cada columna tiene algún valor negativo 2024
negativos = (combined_24.select_dtypes(include=[float, int]) < 0).any()

# Filtrar las columnas que contienen valores negativos (solo mostrar las que son True)
columnas_con_negativos = negativos[negativos]  # Esto te devuelve solo las columnas que son True

print(columnas_con_negativos)


CH06     True
V2_M     True
V3_M     True
V5_M     True
V8_M     True
V9_M     True
V10_M    True
V11_M    True
V12_M    True
V18_M    True
V21_M    True
T_VI     True
dtype: bool


In [42]:
# Verificar si cada columna tiene algún valor negativo
negativos = (combined_24.select_dtypes(include=[float, int]) < 0).any()

# Filtrar las filas que contienen valores negativos (eliminar las filas con cualquier valor negativo)
combined_24_clean = combined_24[~(combined_24.select_dtypes(include=[float, int]) < 0).any(axis=1)]

# Mostrar el DataFrame limpio
print(combined_24_clean)


                             CODUSU  NRO_HOGAR  COMPONENTE  H15  CH03  CH04  \
1     TQRMNOPPQHJNOPCDEIJAH00802435          1           2    1     2     2   
2     TQRMNOPPQHJNOPCDEIJAH00802435          1           4    1     3     1   
3     TQRMNOPPQHKKMMCDEIIAD00858017          1           1    1     1     2   
4     TQRMNOPPQHKMQUCDEIJAH00861670          1           1    1     1     2   
5     TQRMNOPPRHJNOPCDEIJAH00793617          1           1    1     1     2   
...                             ...        ...         ...  ...   ...   ...   
7046  TQUMNOQSVHMOTSCDEIJAH00859221          1           1    1     1     2   
7047  TQUMNOQSVHMOTSCDEIJAH00859221          1           2    1     2     1   
7048  TQVMNORVXHLOTOCDEIJAH00855319          1           1    1     1     2   
7049  TQVMNORVXHLOTOCDEIJAH00855319          1           2    1     3     2   
7050  TQVMNORVXHLOTOCDEIJAH00855319          1           3    1     3     2   

                     CH05  CH06  CH07  CH08  CH09  

In [ ]:
# Verificar si cada columna tiene algún valor negativo 2004 
negativos = (combined_04.select_dtypes(include=[float, int]) < 0).any()

# Filtrar las filas que contienen valores negativos (eliminar las filas con cualquier valor negativo)
combined_04_clean = combined_04[~(combined_04.select_dtypes(include=[float, int]) < 0).any(axis=1)]

# Mostrar el DataFrame limpio
print(combined_04_clean)


In [44]:
# Verificar si cada columna tiene algún valor NaN 2024
nan_values = combined_24.select_dtypes(include=[float, int]).isna().any()

# Filtrar las columnas que contienen valores NaN (solo mostrar las que son True)
columnas_con_nan = nan_values[nan_values]  # Esto te devuelve solo las columnas que tienen NaN

print(columnas_con_nan)


Series([], dtype: bool)


In [ ]:
combined_24 = combined_24.drop(columns = ["CH14", "CH15_COD", "CH16_COD", "IMPUTA", "PP04B3_DIA", "IDECIFR", "PDECIFR", "IDECCFR",'IV1_ESP', 'IV3_ESP', 'IV7_ESP', 'II7_ESP', 'II8_ESP'])

In [45]:
combined_24

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,CH09,CH10,CH11,CH12,CH13,CH15,CH16,NIVEL_ED,ESTADO,CAT_OCUP,CAT_INAC,V2_M,V3_M,V4_M,V5_M,V8_M,V9_M,V10_M,V11_M,V12_M,V18_M,V19_AM,V21_M,T_VI,ANO4,TRIMESTRE,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,IV2,IV3,IV4,IV5,IV6,IV7,IV8,IV9,IV10,IV11,IV12_1,IV12_2,IV12_3,II1,II2,II4_1,II4_2,II4_3,II5,II5_1,II7,II8,II9,V1,V2,V21,V22,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19_A,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,ITF,DECIFR,RDECIFR,GDECIFR,ADECIFR,IPCF,DECCFR,RDECCFR,GDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNOPPQHJNOPCDEIJAH00802435,1,1,1,1,1,1966-07-14 00:00:00,57,2,1,1,2,0,7,2,1,1,5,1,3,0,0,0,0,0,-9,0,0,0,0,0,0,0,-9,2024,1,1,1,S,33,1763,1,5,1,4,1,1,1,1,1,1,2,2,2,2,5,3,1,1,1,2,0,1,1,1,1,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,3,0,3,0,12,12,12.0,12,0.0,12,12,12.0,12,0,2,0,98,0,0,0
1,TQRMNOPPQHJNOPCDEIJAH00802435,1,2,1,2,2,1971-08-17 00:00:00,52,2,1,1,2,0,4,2,1,1,3,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1763,1,5,1,4,1,1,1,1,1,1,2,2,2,2,5,3,1,1,1,2,0,1,1,1,1,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,3,0,3,0,12,12,12.0,12,0.0,12,12,12.0,12,0,2,0,98,0,0,0
2,TQRMNOPPQHJNOPCDEIJAH00802435,1,4,1,3,1,2002-01-04 00:00:00,22,5,1,1,1,1,7,2,1,1,5,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1763,1,5,1,4,1,1,1,1,1,1,2,2,2,2,5,3,1,1,1,2,0,1,1,1,1,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,3,0,3,0,12,12,12.0,12,0.0,12,12,12.0,12,0,2,0,98,0,0,0
3,TQRMNOPPQHKKMMCDEIIAD00858017,1,1,1,1,2,1971-05-09 00:00:00,52,5,2,1,2,0,7,1,1,1,6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,32,702,1,4,1,1,1,1,1,1,1,1,1,2,2,2,4,1,1,1,1,2,0,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2,1,0,1,0,12,12,12.0,12,0.0,12,12,12.0,12,0,1,0,98,0,0,0
4,TQRMNOPPQHKMQUCDEIJAH00861670,1,1,1,1,2,1948-07-24 00:00:00,75,4,1,1,2,0,2,1,1,1,2,1,2,0,105000,0,0,0,0,0,0,0,0,0,0,52000,207000,2024,1,1,1,S,33,2480,1,3,1,4,1,1,1,1,1,1,1,2,2,2,3,1,1,1,1,2,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,2,2,2,1,0,1,347000,4,4,4.0,4,347000.0,8,8,8.0,8,3263,1,0,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7046,TQUMNOQSVHMOTSCDEIJAH00859221,1,1,1,1,2,1962-08-07 00:00:00,61,2,1,1,2,0,2,1,1,1,2,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1103,2,2,1,4,1,1,1,1,1,1,1,2,1,2,2,1,1,2,2,2,0,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2,2,0,2,790000,8,7,8.0,8,395000.0,8,8,8.0,9,1834,1,0,98,0,0,0
7047,TQUMNOQSVHMOTSCDEIJAH00859221,1,2,1,2,1,1971-01-13 00:00:00,53,2,1,1,2,0,2,1,1,1,2,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1103,2,2,1,4,1,1,1,1,1,1,1,2,1,2,2,1,1,2,2,2,0,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2,2,0,2,790000,8,7,8.0,8,395000.0,8,8,8.0,9,1834,1,0,98,0,0,0
7048,TQVMNORVXHLOTOCDEIJAH00855319,1,1,1,1,2,1988-08-22 00:00:00,35,5,1,1,2,0,4,1,1,1,4,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1067,2,1,1,4,1,1,1,1,1,1,1,2,1,2,1,1,2,2,2,0,0,6,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,3,0,3,0,12,12,12.0,12,0.0,12,12,12.0,12,0,1,0,2,0,0,0
7049,TQVMNORVXHLOTOCDEIJAH00855319,1,2,1,3,2,2006-06-26 00:00:00,17,5,1,1,2,0,4,1,1,1,4,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,1,1,1,S,33,1067,2,1,1,4,1,1,1,1,1,1,1,2,1,2,1,1,2,2,2,0,0,6,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,3,0,3,0,12,12,12.0,12,0.0,12,12,12.0,12,0,1,0,2,0,0,0


In [46]:
# Verificar si cada columna tiene algún valor NaN 2004 
nan_values = combined_04.select_dtypes(include=[float, int]).isna().any()

# Filtrar las columnas que contienen valores NaN (solo mostrar las que son True)
columnas_con_nan = nan_values[nan_values]  # Esto te devuelve solo las columnas que tienen NaN

print(columnas_con_nan)


Series([], dtype: bool)


In [30]:
combined_04 = combined_04.drop(columns = "IX_MEN10")

In [47]:
combined_04

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH06,CH07,CH08,CH09,CH10,CH11,CH12,CH13,CH14,CH15,CH15_COD,CH16,CH16_COD,NIVEL_ED,ESTADO,PP06D,PP07E,PP09A,PP09A_ESP,PP09B,PP09C,PP09C_ESP,PP10A,PP10C,PP10D,PP10E,P47T,DECINDR,IDECINDR,RDECINDR,GDECINDR,PDECINDR,ADECINDR,V2_M,V3_M,V4_M,V5_M,V8_M,V9_M,V10_M,V11_M,V12_M,V18_M,V19_AM,V21_M,T_VI,PJ3_1,REALIZADA,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,IV1_ESP,IV2,IV3,IV3_ESP,IV4,IV5,IV6,IV7,IV7_ESP,IV8,IV9,IV10,IV11,IV12_1,IV12_2,IV12_3,II1,II2,II4_1,II4_2,II4_3,II5,II5_1,II7,II7_ESP,II8,II8_ESP,II9,V1,V2,V21,V22,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19_A,V19_B,IX_TOT,IX_MAYEQ10,ITF,DECIFR,IDECIFR,RDECIFR,GDECIFR,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,IDIMPH
0,125047,1.0,1.0,Sí,Jefe,Varón,80.0,Casado,Obra social (incluye PAMI),Sí,"No asiste, pero asistió",0.0,Primario,No,99,En otro país,417,En esta localidad,,Primaria Incompleta (incluye educación especial),Inactivo,0.0,0.0,0.0,...,0.0,0.0,...,0.0,0.0,0.0,0.0,2250.0,10,,10,10,,10,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,2250.0,0.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Partidos del GBA,1703.0,Departamento,,3.0,Mosaico/baldosa/madera/cerámica/alfombra,,Pizarra/teja,Sí,Por cañería dentro de la vivienda,Red pública (agua corriente),,Sí,Dentro de la vivienda,Inodoro con botón/mochila/cadena y arrastre de...,A cámara séptica y pozo ciego,No,No,No,3.0,1.0,Sí,Sí,No,No,0.0,Propietario de la vivienda y el terreno,,Gas de red,,Uso exclusivo del hogar,No,Sí,Sí,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,2.0,2.0,2765.0,10,,10,10,,10,1382.500000,10,,10,10,,10,2.0,0.0,98.0,0.0,0.0,0.0,00000
1,125047,1.0,2.0,Sí,Cónyuge/Pareja,Mujer,75.0,Casado,Obra social (incluye PAMI),Sí,"No asiste, pero asistió",0.0,Primario,No,99,En otro país,417,En esta localidad,,Primaria Incompleta (incluye educación especial),Inactivo,0.0,0.0,0.0,...,0.0,0.0,...,0.0,0.0,0.0,0.0,515.0,06,,06,06,,07,210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,305.0,515.0,0.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Partidos del GBA,1703.0,Departamento,,3.0,Mosaico/baldosa/madera/cerámica/alfombra,,Pizarra/teja,Sí,Por cañería dentro de la vivienda,Red pública (agua corriente),,Sí,Dentro de la vivienda,Inodoro con botón/mochila/cadena y arrastre de...,A cámara séptica y pozo ciego,No,No,No,3.0,1.0,Sí,Sí,No,No,0.0,Propietario de la vivienda y el terreno,,Gas de red,,Uso exclusivo del hogar,No,Sí,Sí,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,2.0,2.0,2765.0,10,,10,10,,10,1382.500000,10,,10,10,,10,2.0,0.0,98.0,0.0,0.0,0.0,00000
2,125098,1.0,1.0,Sí,Jefe,Varón,79.0,Viudo,Obra social (incluye PAMI),Sí,"No asiste, pero asistió",0.0,Secundario,No,03,En esta localidad,,En esta localidad,,Secundaria Incompleta,Inactivo,0.0,0.0,0.0,...,0.0,0.0,...,0.0,0.0,0.0,0.0,368.0,05,,04,04,,03,368.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,368.0,0.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,Departamento,,3.0,Mosaico/baldosa/madera/cerámica/alfombra,,Membrana/cubierta asfáltica,Sí,Por cañería dentro de la vivienda,Red pública (agua corriente),,Sí,Dentro de la vivienda,Inodoro con botón/mochila/cadena y arrastre de...,A red pública (cloaca),No,No,No,3.0,1.0,Sí,No,No,No,0.0,Otra situación,CEDIDO A SUS HIJAS CON CLAUSULA DE USO HASTA,Gas de red,,Compartido con otro/s hogar/es de la misma viv...,No,Sí,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1.0,1.0,368.0,02,,02,02,,01,368.000000,07,,06,06,,04,1.0,0.0,98.0,0.0,0.0,0.0,00000
3,125169,1.0,1.0,Sí,Jefe,Varón,72.0,Unido,No paga ni le descuentan,Sí,"No asiste, pero asistió",0.0,Primario,No,06,En otra provincia (especificar),034,En esta localidad,,Primaria Incompleta (incluye educación especial),Ocupado,0.0,...un plan de empleo?,Partidos del GBA,...,0.0,0.0,...,0.0,0.0,0.0,0.0,150.0,02,,02,02,,02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sí,2004.0,1er. Trimestre,G

#### Valores Faltantes
- 2004 no tiene
- 2024 se eliminaron los más brutos y se imputaron por la mediana los menos

##### 2004
-Tenia una columna con valores Nan = IX_MEN10



##### 2024
- Tiene columnas con valores Nan
- Se revisa la proporción de Nan que hay por columna
- Elimino las columnas que tienen proporción de Nans > 90
- Imputo por la mediana en las columnas que su proporción de Nans < 90

In [ ]:
# # Obtengo las columnas que contienen datos faltantes, reviso que proporción de los valores faltas

# # Porcentaje de valores faltantes por columna
# lista_nans_poco = []
# lista_nans_50 = []
# lista_nans_60 = []
# lista_nans_70 = []
# lista_nans_80 = []
# lista_nans_90 = []
# lista_nans_100 = []

# for i in columnas_con_nan:
#     proporcion_columna = combined_24[i].isnull().sum() / len(combined_24) * 100
    
#     if proporcion_columna < 50:
#         lista_nans_poco.append(i)

#     elif proporcion_columna >= 50 and proporcion_columna < 60:
#         lista_nans_50.append(i)

#     elif proporcion_columna >= 60 and proporcion_columna < 70:
#         lista_nans_60.append(i)

#     elif proporcion_columna >= 70 and proporcion_columna < 80:
#         lista_nans_70.append(i)

#     elif proporcion_columna >= 80 and proporcion_columna < 90:
#         lista_nans_80.append(i)

#     elif proporcion_columna >= 90 and proporcion_columna < 100:
#         lista_nans_90.append(i)

#     elif proporcion_columna == 100:
#         lista_nans_100.append(i)
    


################################# SE PODRIA ELIMINAR ###########################################

In [ ]:
# Elimino las variables que tienen más de 90% de datos Nan. Me fijo antes cuantas columnas son
columnas_elim = lista_nans_90 + lista_nans_100
combined_24 = combined_24.drop(columns = (columnas_elim))

# Imputo por la mediana en las columnas que su proporción de Nans < 90
columnas_ok = lista_nans_poco + lista_nans_50 +lista_nans_60 + lista_nans_70 + lista_nans_80
combined_24 = combined_24[columnas_ok].fillna(combined_24[columnas_ok].median()) #ACA HAY ALGO RARO QUE HACE QUE QUEDEN TRES COLUMNAS EN EL DF!!



#### 
# COMENTARIO MARCOS 24/11 A LAS 15 HORAS 
# CREO QUE ESTO LO PODEMOS SACAR 
# SE VUELVE MEDIO UN QUILOMBO Y NOS TERMINA CAGANDO Y NOS DEJA CON 3 VARIABLES 
# PARA MI DEJAMOS COMO ARRIBA TODO ELIMINADO LO QUE TENGAN NANS Y LISTO 

#####
#FIN DEL COMUNICADO

################################# SE PODRIA ELIMINAR ###########################################


In [47]:
columnas_ok

['CH14', 'PP04B3_DIA', 'CH15_COD']

In [ ]:
combined_24

##### Outliers 

In [ ]:
# Outliers
# COMBINED 04
import pandas as pd

# Supongamos que tienes un DataFrame llamado combined_04
# Primero seleccionamos solo las columnas numéricas para el análisis
numerical_columns = combined_04.select_dtypes(include=['float64', 'int64']).columns

# Calculamos los cuartiles y el IQR solo para las columnas numéricas
Q1 = combined_04[numerical_columns].quantile(0.25)
Q3 = combined_04[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Detectar outliers (donde los valores están fuera de los límites)
outliers = (combined_04[numerical_columns] < lower_limit) | (combined_04[numerical_columns] > upper_limit)

# Mostrar una vista de los outliers por fila
outliers_data = combined_04[outliers.any(axis=1)]

# Opcional: Mostrar el total de outliers por columna
outliers_count = outliers.sum()
print(f"Total de outliers por columna: \n{outliers_count}")

# Mostrar las filas que contienen outliers
print(f"Filas con outliers: \n{outliers_data}")

# Eliminar filas con outliers
combined_04_no_outliers = combined_04[~outliers.any(axis=1)]
print(f"Datos después de eliminar outliers: \n{combined_04_no_outliers.head()}")

Total de outliers por columna: 
NRO_HOGAR      200
PP06D          185
P47T           496
V2_M           764
V3_M             8
V4_M            13
V5_M           107
V8_M            71
V9_M             8
V10_M           15
V11_M           13
V12_M          229
V18_M            5
V19_AM           0
V21_M          107
T_VI          1118
ANO4             0
PONDERA        363
II5_1           76
IX_TOT         324
IX_MAYEQ10     194
ITF            510
IPCF           550
dtype: int64
Filas con outliers: 
        CODUSU  NRO_HOGAR COMPONENTE H15            CH03   CH04  CH06  \
0     125047          1.0        1.0  Sí            Jefe  Varón  80.0   
1     125047          1.0        2.0  Sí  Cónyuge/Pareja  Mujer  75.0   
2     125098          1.0        1.0  Sí            Jefe  Varón  79.0   
9     125172          2.0        1.0  Sí            Jefe  Varón  55.0   
13    125295          1.0        1.0  Sí            Jefe  Varón  34.0   
...        ...        ...        ...  ..             ...   

##### 2024

In [ ]:
# COMBINED 24
import pandas as pd

# Supongamos que tienes un DataFrame llamado combined_04
# Primero seleccionamos solo las columnas numéricas para el análisis
numerical_columns = combined_24.select_dtypes(include=['float64', 'int64']).columns

# Calculamos los cuartiles y el IQR solo para las columnas numéricas
Q1 = combined_24[numerical_columns].quantile(0.25)
Q3 = combined_24[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Detectar outliers (donde los valores están fuera de los límites)
outliers = (combined_24[numerical_columns] < lower_limit) | (combined_24[numerical_columns] > upper_limit)

# Mostrar una vista de los outliers por fila
outliers_data = combined_24[outliers.any(axis=1)]

# Opcional: Mostrar el total de outliers por columna
outliers_count = outliers.sum()
print(f"Total de outliers por columna: \n{outliers_count}")

# Mostrar las filas que contienen outliers
print(f"Filas con outliers: \n{outliers_data}")

# Eliminar filas con outliers
#combined_04_no_outliers = combined_04[~outliers.any(axis=1)]
#print(f"Datos después de eliminar outliers: \n{combined_04_no_outliers.head()}")

##### Variables categoricas

##### 2004

In [ ]:
# Variables categoricas 04
# origin_dummies = pd.get_dummies(auto['origin'], prefix='origin')
# # Concatenamos con el df original
# auto_d = pd.concat([auto, origin_dummies], axis=1)


# # CH05 (fecha de nacimiento)
# # CH06 
# # PP03D
# # PP3E_TOT
# # PP3F_TOT
# # PP04B2
# # PP04B3_MES
# # PP04B3_ANO

In [ ]:
# # Paso 1: Identificar las columnas numéricas continuas
# numerical_columns = combined_04.select_dtypes(include=['float64', 'int64']).columns
# print("Columnas numéricas continuas:")
# print(numerical_columns)

# # Paso 2: Identificar las columnas categóricas
# categorical_columns = combined_04.select_dtypes(include=['object', 'category']).columns
# print("Columnas categóricas:")
# print(categorical_columns)

# # Paso 3: Convertir las columnas categóricas en variables dummies
# combined_04_dummies = pd.get_dummies(combined_04, columns=categorical_columns, drop_first=True)

# # Mostrar el DataFrame resultante
# print("DataFrame con variables dummy:")
# print(combined_04_dummies.head())

Columnas numéricas continuas:
Index(['NRO_HOGAR', 'ANO4', 'PONDERA', 'PP03D', 'PP3E_TOT', 'PP3F_TOT',
       'PP04B2', 'PP04B3_MES', 'PP04B3_ANO', 'PP04B3_DIA', 'PP05B2_MES',
       'PP05B2_ANO', 'PP05B2_DIA', 'PP06C', 'PP06D', 'PP08D1', 'PP08D4',
       'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'PP11B2_MES',
       'PP11B2_ANO', 'PP11B2_DIA', 'PP11G_ANO', 'PP11G_MES', 'PP11G_DIA',
       'P21', 'TOT_P12', 'P47T', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M',
       'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI',
       'ITF', 'IPCF'],
      dtype='object')
Columnas categóricas:
Index(['CODUSU', 'COMPONENTE', 'H15', 'TRIMESTRE', 'REGION', 'MAS_500',
       'AGLOMERADO', 'CH03', 'CH04', 'CH06',
       ...
       'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR',
       'PJ1_1', 'PJ2_1', 'PJ3_1', 'IDIMPP'],
      dtype='object', length=130)
DataFrame con variables dummy:
   NRO_HOGAR    ANO4  PONDERA  PP03D  PP3E_TOT  PP3F_TOT  PP04B2  PP04B3_MES  \

In [ ]:
# # Convertir una columna numérica discreta en categórica (si tiene pocos valores únicos)
# combined_04['H15_x'] = combined_04['H15_x'].astype('category')

# # Identificar las columnas categóricas después de la conversión
# categorical_columns = combined_04.select_dtypes(include=['object', 'category']).columns
# print("Columnas categóricas después de la conversión:")
# print(categorical_columns)

##### 2024

In [ ]:
lista = combined_04.columns.to_list()

# numericas = ["CH05","CH06","PP03D","PP3E_TOT","PP3F_TOT","PP04B2","PP04B3_MES","PP04B3_ANO","PP04B3_DIA","CODUSU","NRO_HOGAR","IV2","IV1_Esp","IV7_Esp","II2", "II4","IDIMPH", "PP04G_COD", "IX_TOT", "IX_MEN10", "IX_MAYEQ10", "ITF", "DECIFR", "IDECIFR", "RDECIFR", "GDECIFR", "PDECIFR", "ADECIFR", "IPCF", "DECCFR", "IDECCFR", "RDECCFR", "GDECCFR", "PDECCFR", "ADECCFR", "PP06C", "PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3", "PP09A_ESP", "PP09C_ESP", "PP11D_COD", "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR", "TOT_P12", "P47T", "DECINDR", "IDECINDR", "RDECINDR", "GDECINDR", "PDECINDR", "ADECINDR","V2_M", "V3_M", "V4_M", "V5_M", "V8_M", "V9_M", "V10_M", "V11_M", "V12_M", "V18_M", "V19_M", "V21_M", "T_VI", "ITF", "DECIFR", "IDECIFR", "RDECIFR", "GDECIFR", "PDECIFR", "ADECIFR", "ADECIFR", "IDIMPP"]
# numericas = list(set(numericas))

# for item in numericas:
#     if item in lista:
#         lista.remove(item)

# CH05 (fecha de nacimiento)
# CH06 
# PP03D
# PP3E_TOT
# PP3F_TOT
# PP04B2
# PP04B3_MES
# PP04B3_ANO

In [15]:
reemplazos = {
    "Sí": 1, 
    "No" : 0, 
    "S" : 1, 
    "N": 0
}

combined_04_bin = combined_04.applymap(lambda x: reemplazos.get(x, x))

C:\Users\magda\AppData\Local\Temp\ipykernel_28284\2313058199.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_04_bin = combined_04.applymap(lambda x: reemplazos.get(x, x))


In [16]:
combined_04_bin

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH06,CH07,CH08,CH09,...,GDECCFR,PDECCFR,ADECCFR,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,IDIMPH
0,125047,1.0,1.0,1.0,Jefe,Varón,80.0,Casado,Obra social (incluye PAMI),1,...,10,,10,2.0,0.0,98.0,0.0,0.0,0.0,00000
1,125047,1.0,2.0,1.0,Cónyuge/Pareja,Mujer,75.0,Casado,Obra social (incluye PAMI),1,...,10,,10,2.0,0.0,98.0,0.0,0.0,0.0,00000
2,125098,1.0,1.0,1.0,Jefe,Varón,79.0,Viudo,Obra social (incluye PAMI),1,...,06,,04,1.0,0.0,98.0,0.0,0.0,0.0,00000
3,125169,1.0,1.0,1.0,Jefe,Varón,72.0,Unido,No paga ni le descuentan,1,...,01,,01,2.0,0.0,3.0,0.0,0.0,0.0,00000
4,125169,1.0,2.0,1.0,Cónyuge/Pareja,Mujer,66.0,Unido,No paga ni le descuentan,1,...,01,,01,2.0,0.0,3.0,0.0,0.0,0.0,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7642,288171,1.0,3.0,0.0,Hijo/Hijastro,Mujer,1.0,Soltero,Mutual/Prepaga/Servicio de emergencia,Menor de 2 años,...,08,,09,Servicio doméstico,0.0,2.0,0.0,0.0,0.0,00000
7643,288405,1.0,1.0,1.0,Jefe,Varón,51.0,Casado,Obra social (incluye PAMI),1,...,02,,02,2.0,0.0,98.0,0.0,0.0,0.0,00000
7644,288405,1.0,2.0,1.0,Cónyuge/Pareja,Mujer,55.0,Casado,Obra social (incluye PAMI),1,...,02,,02,2.0,0.0,98.0,0.0,0.0,0.0,00000
7645,288405,1.0,3.0,1.0,Hijo/Hijastro,Mujer,12.0,Soltero,Obra social (incluye PAMI),1,...,02,,02,2.0,0.0,98.0,0.0,0.0,0.0,00000


In [17]:
combined_24

,CH14,PP04B3_DIA,CH15_COD
0,3.0,0.0,86.0
1,3.0,0.0,86.0
2,0.0,0.0,86.0
3,2.0,0.0,86.0
4,2.0,0.0,86.0
...,...,...,...
7046,2.0,0.0,86.0
7047,2.0,0.0,86.0
7048,2.0,0.0,86.0
7049,2.0,0.0,86.0


In [ ]:
nobinarias = [col for col in combined_04.columns if combined_04[col].nunique()> 2]

In [39]:
combined_04.describe().T

,count,mean,std,min,25%,50%,75%,max
NRO_HOGAR,7647.0,1.026808,0.165531,1.0,1.0,1.000000,1.0,3.0
PP06D,7647.0,20.033346,201.774673,-8.0,0.0,0.000000,0.0,10000.0
P47T,7647.0,362.510658,897.852758,0.0,0.0,40.000000,500.0,54000.0
V2_M,7647.0,44.906761,216.244999,0.0,0.0,0.000000,0.0,9999.0
V3_M,7647.0,0.487773,24.327418,0.0,0.0,0.000000,0.0,2000.0
V4_M,7647.0,0.397803,9.972391,0.0,0.0,0.000000,0.0,375.0
V5_M,7647.0,2.209363,19.914964,0.0,0.0,0.000000,0.0,375.0
V8_M,7647.0,4.473650,70.854853,0.0,0.0,0.000000,0.0,3500.0
V9_M,7647.0,1.098470,50.404813,0.0,0.0,0.000000,0.0,4000.0
V10_M,7647.0,0.800706,29.280218,0.0,0.0,0.000000,0.0,1700.0


In [15]:
combined_04.nunique()> 2

CODUSU         True
NRO_HOGAR      True
COMPONENTE     True
H15            True
ANO4          False
              ...  
ADECCFR        True
PJ1_1         False
PJ2_1         False
PJ3_1         False
IDIMPP         True
Length: 176, dtype: bool

In [ ]:
(combined_04["PP08F1"])

2700.0

In [ ]:
categoricas = [elemento for elemento in nobinarias if elemento not in numericas]

In [ ]:
# categoricas04 = ["IV1", "IV3", "IV4", "IV5", "IV6", "IV7", "I"]

In [ ]:
len(categoricas)

101

In [ ]:
len(numericas)

75

In [ ]:
cantbinarias = len(lista) - len(nobinarias)
cantbinarias

21

In [ ]:
categoricas
for col in categoricas:
    dummies = pd.get_dummies(combined_04_bin[col], prefix=col)
    combined_04_cat = pd.concat([combined_04_bin, dummies], axis = 1)

combined_04_cat 

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,GDECCFR,PDECCFR,ADECCFR,PJ1_1,PJ2_1,PJ3_1,IDIMPP,PP11T_0.0,PP11T_1,PP11T_Ns./Nr.
0,125047,1.0,1.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1703.0,...,10,,10,0.0,0.0,0.0,00000,True,False,False
1,125047,1.0,2.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1703.0,...,10,,10,0.0,0.0,0.0,00000,True,False,False
2,125047,1.0,1.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1703.0,...,10,,10,0.0,0.0,0.0,00000,True,False,False
3,125047,1.0,2.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1703.0,...,10,,10,0.0,0.0,0.0,00000,True,False,False
4,125098,1.0,1.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Ciudad de Buenos Aires,1299.0,...,06,,04,0.0,0.0,0.0,00000,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32906,288405,1.0,4.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1673.0,...,02,,02,0.0,0.0,0.0,00000,True,False,False
32907,288405,1.0,1.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1673.0,...,02,,02,0.0,0.0,0.0,00000,True,False,False
32908,288405,1.0,2.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1673.0,...,02,,02,0.0,0.0,0.0,00000,True,False,False
32909,288405,1.0,3.0,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,Partidos del GBA,1673.0,...,02,,02,0.0,0.0,0.0,00000,True,False,False


In [ ]:
combined_04_cat.head()

,CODUSU,NRO_HOGAR,ANO4,TRIMESTRE,REGION,MAS_500,CH06,PP03D,PP3E_TOT,PP3F_TOT,...,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,IDIMPP,PJ3_1_0.0,PJ3_1_Tiene plan jefas y jefes,PJ3_1_Ns./Nr.
0,125047,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,80.0,0.0,0.0,0.0,...,10,,10,10,,10,00000,True,False,False
1,125047,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,75.0,0.0,0.0,0.0,...,10,,10,10,,10,00000,True,False,False
2,125047,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,80.0,0.0,0.0,0.0,...,10,,10,10,,10,00000,True,False,False
3,125047,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,75.0,0.0,0.0,0.0,...,10,,10,10,,10,00000,True,False,False
4,125098,1.0,2004.0,1er. Trimestre,Gran Buenos Aires,1,79.0,0.0,0.0,0.0,...,07,,06,06,,04,00000,True,False,False


In [ ]:
#DEFINIR VARIABLES EXTRA

#Desocupado dummy

#NSE - con ingreso. agrupar variables

#subsidios V5, V12, V19_A (MENORES DE 10 AÑOS)

## Clasificacion y regularizacion

In [ ]:
#FILTRAR PERSONAS Q NO RESPONDIERON ESTADO=0
#ESTADO: 0 = Entrevista individual no realizada (no respuesta al cuestionario individual)
    # 1 = Ocupado
    # 2 = Desocupado
    # 3 = Inactivo
    # 4 = Menor de 10 años

In [ ]:
#ACA HAY QUE DEFINIR LA BASE DE RESPONDIERON Y NO RESPONDIERON PARA CADA AÑO

In [ ]:
# Se separa X e y a partir de la selección de columnas numericas
#ESTO HAY QUE UPDATEARLO CON LAS BASES FINALES
X_2004 = again_04_selec.drop(columns=['desocupado'])
y_2004 = again_04_selec['desocupado']
X_2024 = again_24_selec.drop(columns=['desocupado'])
y_2024 = again_24_selec['desocupado']

In [ ]:
#Partir la base train test para cada ano. 
#Semilla 101
#agregar columna de unos (1)
x_train_24, x_test_24, y_train_24, y_test_24 = train_test_split(X_2024, y_2024, test_size=0.3, random_state=101)
x_train_04, x_test_04, y_train_04, y_test_04 = train_test_split(X_2004, y_2004, test_size=0.3, random_state=101)

x_train_04 = sm.add_constant(x_train_04) #AGREGO COLUMNA DE UNOS
x_train_24 = sm.add_constant(x_train_24)

In [ ]:
#IMPORTANTE ESTANDARIZAR ANTES DE USAR METODOS DE REGULARIZACION
# Iniciamos el Standard Scaler
sc = StandardScaler()

# Estandarizamos las observaciones de entrenamiento
x_train_04_transformed = pd.DataFrame(sc.fit_transform(x_train_04), index=x_train_04.index, columns=x_train_04.columns)
x_train_24_transformed = pd.DataFrame(sc.fit_transform(x_train_24), index=x_train_24.index, columns=x_train_24.columns)

# Estandarizamos las observaciones de test
x_test_04_transformed = pd.DataFrame(sc.transform(x_test_04), index=x_test_04.index, columns=x_test_04.columns)
x_test_24_transformed = pd.DataFrame(sc.transform(x_test_24), index=x_test_24.index, columns=x_test_24.columns)  

# Estadisticas luego de estandarizar
x_train_04_transformed.describe().T
x_train_24_transformed.describe().T

In [ ]:
#Regresion logistica con penalidad LASSO (todos estos pasos se hacen para cada ano)
#### Regresion logistica para 2004
alpha = 1
log04_lasso = LogisticRegression(penalty='l1', C=(1/alpha), solver='liblinear').fit(x_train_04, y_train_04)
y_test_pred_score_04_lasso = log04_lasso.predict_proba(x_train_04)[:,1]
y_pred_log04_lasso = log04_lasso.predict(x_test_04)

#### Regresion logistica para 2024
log24_lasso = LogisticRegression(penalty='l1', C=(1/alpha), solver='liblinear').fit(x_train_24, y_train_24)
y_test_pred_score_24_lasso = log24_lasso.predict_proba(x_train_24)[:,1]
y_pred_log24_lasso = log24_lasso.predict(x_test_24)

NameError: name 'x_train_04' is not defined

In [ ]:
#### Medidas de precision Regresion logistica LASSO para 2004
#Matriz de confusion
conflog04_lasso = confusion_matrix(y_test_04, y_pred_log04_lasso) 
print("Matriz de confusion (Reg Log LASSO 2004)\n", conflog04_lasso)

#AUC
auc_log04_lasso = roc_auc_score(y_test_04, y_pred_log04_lasso)
print('\nAUC Reg Log (2004): %.4f' %auc_log04_lasso)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_04, y_pred_log04_lasso)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log04_lasso, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_l04 = mean_squared_error(y_test_04, y_pred_log04_lasso)
print("Error cuadrático medio:", ecm_a1_l04) 

#Accuracy
accuracy_log04_lasso = accuracy_score(y_test_04, y_pred_log04_lasso)
print("La accuracy del modelo (Reg Log LASSO 2004) es: %.4f" %accuracy_log04_lasso)

In [ ]:
#### Medidas de precision Regresion logistica LASSO para 2024
#Matriz de confusion
conflog24_lasso = confusion_matrix(y_test_24, y_pred_log24_lasso) 
print("Matriz de confusion (Reg Log LASSO 2024)\n", conflog24_lasso)

#AUC
auc_log24_lasso = roc_auc_score(y_test_24, y_pred_log24_lasso)
print('\nAUC Reg Log LASSO (2024): %.4f' %auc_log24_lasso)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_24, y_pred_log24_lasso)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log24_lasso, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_l24 = mean_squared_error(y_test_24, y_pred_log24_lasso)
print("Error cuadrático medio:", ecm_a1_l24) 

#Accuracy
accuracy_log24_lasso = accuracy_score(y_test_24, y_pred_log24_lasso)
print("La accuracy del modelo (Reg Log LASSO 2024) es: %.4f" %accuracy_log24_lasso)

In [ ]:
#Regresion logistica con penalidad ridge (todos estos pasos se hacen para cada ano)
#### Regresion logistica para 2004
alpha = 1
log04_ridge = LogisticRegression(penalty='l2', C=(1/alpha), solver='liblinear').fit(x_train_04, y_train_04)
y_test_pred_score_04_ridge = log04_ridge.predict_proba(x_train_04)[:,1]
y_pred_log04_ridge = log04_ridge.predict(x_test_04)

#### Regresion logistica para 2024
log24_ridge = LogisticRegression(penalty='l2', C=(1/alpha), solver='liblinear').fit(x_train_24, y_train_24)
y_test_pred_score_24_ridge = log24_ridge.predict_proba(x_train_24)[:,1]
y_pred_log24_ridge = log24_ridge.predict(x_test_24)

In [ ]:
#### Medidas de precision Regresion logistica ridge para 2004
#Matriz de confusion
conflog04_ridge = confusion_matrix(y_test_04, y_pred_log04_ridge) 
print("Matriz de confusion (Reg Log ridge 2004)\n", conflog04_ridge)

#AUC
auc_log04_ridge = roc_auc_score(y_test_04, y_pred_log04_ridge)
print('\nAUC Reg Log ridge (2004): %.4f' %auc_log04_ridge)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_04, y_pred_log04_ridge)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log04_ridge, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_r04 = mean_squared_error(y_test_04, y_pred_log04_ridge)
print("Error cuadrático medio:", ecm_a1_r04)  

#Accuracy
accuracy_log04_ridge = accuracy_score(y_test_04, y_pred_log04_ridge)
print("La accuracy del modelo (Reg Log ridge 2004) es: %.4f" %accuracy_log04_ridge)

In [ ]:
#### Medidas de precision Regresion logistica ridge para 2024
#Matriz de confusion
conflog24_ridge = confusion_matrix(y_test_24, y_pred_log24_ridge) 
print("Matriz de confusion (Reg Log ridge 2024)\n", conflog04_ridge)

#AUC
auc_log24_ridge = roc_auc_score(y_test_24, y_pred_log24_ridge)
print('\nAUC Reg Log ridge (2024): %.4f' %auc_log24_ridge)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_24, y_pred_log24_ridge)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log24_ridge, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_r24 = mean_squared_error(y_test_24, y_pred_log24_ridge)
print("Error cuadrático medio:", ecm_a1_r24) 

#Accuracy
accuracy_log24_ridge = accuracy_score(y_test_24, y_pred_log24_ridge)
print("La accuracy del modelo (Reg Log ridge 2024) es: %.4f" %accuracy_log24_ridge)

In [ ]:
# Cross validation para elegir el alpha para los metodos de regularizacion
from sklearn.linear_model import LogisticRegressionCV

#LASSO 2004
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_lasso_cv_04 = LogisticRegressionCV(Cs=10, cv=10, penalty='l1', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_lasso_cv_04.fit(x_train_04, y_train_04)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_lasso_cv_04.C_[0])

print("El mejor valor de alpha:", (1/log_lasso_cv_04.C_[0]))

# # Hacer predicciones (opcional)
# y_pred_ = log_lasso_cv_04.predict(x_test_04)

#LASSO 2024
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_lasso_cv_24 = LogisticRegressionCV(Cs=10, cv=10, penalty='l1', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_lasso_cv_24.fit(x_train_24, y_train_24)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_lasso_cv_24.C_[0])

print("El mejor valor de alpha:", (1/log_lasso_cv_24.C_[0]))

In [ ]:
#Ridge 2004
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_ridge_cv_04 = LogisticRegressionCV(Cs=10, cv=10, penalty='l2', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_ridge_cv_04.fit(x_train_04, y_train_04)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_ridge_cv_04.C_[0])

print("El mejor valor de alpha:", (1/log_ridge_cv_04.C_[0]))

# # Hacer predicciones (opcional)
# y_pred_ = log_lasso_cv_04.predict(x_test_04)

#Ridge 2024
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_ridge_cv_24 = LogisticRegressionCV(Cs=10, cv=10, penalty='l2', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_ridge_cv_24.fit(x_train_24, y_train_24)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_ridge_cv_24.C_[0])

print("El mejor valor de alpha:", (1/log_ridge_cv_24.C_[0]))

In [ ]:
#boxplot usando seaborn: distribucion del error de prediccion para cada alpha

#lineplot para lasso: promedio de la proporcion de variables ignoradas por el modelo 

In [ ]:
#MODELOS LASSO OPTIMOS

#2004

#2024

In [ ]:
#MODELOS RIDGE OPTIMOS

#2004

#2024

In [ ]:
#comparar resultados lasso y ridge 2004 vs 2024